In [ ]:

# -*- coding: utf-8 -*-
"""
LSTM to predict a sine wave (TensorFlow/Keras)
"""

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# 1) Generate a sine wave
np.random.seed(42)
n_points = 2000
t = np.linspace(0, 50 * np.pi, n_points)  # 50 periods
signal = np.sin(t)

# Optional: add a tiny bit of noise to make learning less trivial
noise_level = 0.0
signal_noisy = signal + noise_level * np.random.randn(n_points)

# 2) Scale (LSTMs often do better with normalized inputs)
scaler = MinMaxScaler(feature_range=(-1, 1))
signal_scaled = scaler.fit_transform(signal_noisy.reshape(-1, 1)).flatten()

# 3) Create supervised sequences
def make_sequences(series, window_size=50, horizon=1):
    """
    window_size: number of past steps to use
    horizon: predict horizon steps ahead (default 1 step ahead)
    Returns X of shape (num_samples, window_size, 1) and y of shape (num_samples,)
    """
    X, y = [], []
    for i in range(len(series) - window_size - horizon + 1):
        X.append(series[i : i + window_size])
        y.append(series[i + window_size + horizon - 1])
    X = np.array(X)
    y = np.array(y)
    return X[..., np.newaxis], y  # add feature dimension

WINDOW_SIZE = 64
HORIZON = 1

X, y = make_sequences(signal_scaled, window_size=WINDOW_SIZE, horizon=HORIZON)

# Train/validation split
split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_val, y_val = X[split:], y[split:]

# 4) Build the LSTM model
model = Sequential([
    LSTM(64, input_shape=(WINDOW_SIZE, 1), return_sequences=False),
    Dense(32, activation='tanh'),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

model.summary()

# 5) Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    verbose=1
)

# 6) Predict on the validation set
y_val_pred = model.predict(X_val)

# Inverse scale back to original sine range
y_val_inv = scaler.inverse_transform(y_val.reshape(-1, 1)).flatten()
y_val_pred_inv = scaler.inverse_transform(y_val_pred.reshape(-1, 1)).flatten()

# 7) Plot
plt.figure(figsize=(12, 5))
plt.plot(range(len(signal)), signal, color='lightgray', label='True sine (full)')
val_start = split + WINDOW_SIZE + HORIZON - 1
plt.plot(range(val_start, val_start + len(y_val_inv)), y_val_inv, label='True (val)', color='tab:blue')
plt.plot(range(val_start, val_start + len(y_val_pred_inv)), y_val_pred_inv, label='Predicted (val)', color='tab:orange')
plt.title("LSTM Sine Wave Prediction (Keras)")
plt.xlabel("Time step")
plt.ylabel("Amplitude")
plt.legend()
plt.tight_layout()
plt.show()

# Optional: one-step-ahead rolling forecast over entire series
def rolling_forecast(model, series_scaled, window_size, steps):
    """Generate iterative one-step-ahead forecasts."""
    preds = []
    window = series_scaled[:window_size].copy()
    for i in range(steps):
        x = window.reshape(1, window_size, 1)
        yhat = model.predict(x, verbose=0)[0, 0]
        preds.append(yhat)
        # shift window and append prediction (teacher forcing alternative)
        window = np.roll(window, -1)
        window[-1] = yhat
    return np.array(preds)

rf_preds_scaled = rolling_forecast(model, signal_scaled, WINDOW_SIZE, steps=n_points - WINDOW_SIZE)
rf_preds = scaler.inverse_transform(rf_preds_scaled.reshape(-1, 1)).flatten()

plt.figure(figsize=(12, 5))
plt.plot(signal, label="True sine")
plt.plot(range(WINDOW_SIZE, WINDOW_SIZE + len(rf_preds)), rf_preds, label="Rolling forecast", color="tab:red")
plt.title("LSTM Rolling Forecast on Sine (Keras)")
plt.xlabel("Time step")
plt.ylabel("Amplitude")
plt.legend()
plt.tight_layout()
plt.show()
